In [22]:
# 导入包文件
from tqdm import tqdm
import numpy as np
import time
import datetime# 获取数据
from download_data_export_thailand import downloadexport
import download_data_export_thailand


In [23]:
# 下载数据
url = "https://dataapi.moc.go.th/export-harmonize-countries"
a = downloadexport(url=url,hs_code="081060",star_year=2020,star_month=1,end_year=2021,end_month=1)
a.to_excel()
data1 = a.down_data()
data = data1  


In [39]:
# index转换为datetime格式
date = [datetime.datetime.strptime(i,'%Y/%m') for i in data.index.values]
date = [i.strftime('%Y%mM') for i in date]
data['date'] = date
data.set_index(keys = 'date',inplace = True)
data

,country_code,country_name_en,quantity,acc_quantity,value_usd,acc_value_usd,value_baht,acc_value_baht
date,,,,,,,,
202001M,CN,CHINA,2774418.0,2774418.0,7175459.0,7175459.0,214796646.0,214796646.0
202001M,HK,HONG KONG,1545696.0,1545696.0,6638127.0,6638127.0,198711673.0,198711673.0
202001M,VN,VIETNAM,1999461.0,1999461.0,6501256.0,6501256.0,194614437.0,194614437.0
202001M,US,U.S.A.,13517.0,13517.0,135941.0,135941.0,4069371.0,4069371.0
202001M,CA,CANADA,3040.0,3040.0,34932.0,34932.0,1045684.0,1045684.0
...,...,...,...,...,...,...,...,...
202101M,SE,SWEDEN,97.0,97.0,405.0,405.0,12083.0,12083.0
202101M,BH,BAHRAIN,80.0,80.0,369.0,369.0,10996.0,10996.0
202101M,CZ,CZECH REPUBLIC,78.0,78.0,312.0,312.0,9295.0,9295.0


In [40]:
# 数据分类
data_all = data.groupby(data.index).sum()
data_china = data[data['country_name_en'] == 'CHINA']
data_hk = data[data['country_name_en'] == 'HONG KONG']
data_else_country = data[~(data['country_name_en'].str.contains('CHINA') | data['country_name_en'].str.contains('HONG KONG'))]
# 将国外数据分类
data_else_country = data_else_country.groupby(data_else_country.index).sum()
a = list(map(lambda x:round(x/17280,0),data_all.quantity.values.tolist()))
a

[368.0,
 437.0,
 3523.0,
 9144.0,
 8736.0,
 2677.0,
 4133.0,
 1876.0,
 3464.0,
 834.0,
 407.0,
 332.0,
 532.0]

In [41]:
# 绘图
import pyecharts.options as opts
from pyecharts.charts import Line

c = (
    Line()
    # 
    
    .add_xaxis(data_china.index.values.tolist())
    .add_yaxis(
        "ALL",
        list(map(lambda x:round(x/17280,0),data_all.quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),

    )
    .add_yaxis(
        "CHINA",
        list(map(lambda x:round(x/17280,0),data_china[data_china.country_name_en == 'CHINA'].quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),

    )
    .add_yaxis(
        "HONG KONG",
        list(map(lambda x:round(x/17280,0),data_hk[data_hk.country_name_en == 'HONG KONG'].quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
    )
    .add_yaxis(
        "ELSE COUNTRY",
        list(map(lambda x:round(x/17280,0),data_else_country.quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="2020-2021年泰国榴莲出口数据（柜/月）"))
    .render(r"line_markline.html")
)
